In [7]:
from __future__ import absolute_import, division, print_function

In [8]:
import tensorflow as tf
from dataProcessing import readAfterDSPDataInTxt,readSourceDataFromVPI
import numpy as np
rng = np.random

In [9]:
# Parameters.
learning_rate = 0.01
training_steps = 1000
display_step = 50

In [10]:
#Load data
data_x=readAfterDSPDataInTxt('afterDSP_x.txt')
# data_y=readAfterDSPDataInTxt('afterDSP_y.txt')
resouce_x=readSourceDataFromVPI('bits_X.txt')
# resouce_y=readSourceDataFromVPI('bits_Y.txt')
# print(resouce_x.shape)
print(data_x)

#将复数数组转换成[[r,i]]数组
data_ri_x=np.stack((data_x.real,data_x.imag),-1)
source_ri_x=np.stack((resouce_x.real,resouce_x.imag),-1)
print(data_ri_x)
print(source_ri_x)

# Load a numpy array using tf data api with `from_tensor_slices`.
data = tf.data.Dataset.from_tensor_slices((data_ri_x, source_ri_x))
# Refill data indefinitely.  
data = data.repeat()
# Shuffle data.
data = data.shuffle(buffer_size=100)
# Batch data (aggregate records together).
data = data.batch(batch_size=4)
# Prefetch batch (pre-load batch for faster consumption).
data = data.prefetch(buffer_size=1)

[-2.90141193+2.63064032j -2.81521769-1.06264591j  1.02505341-0.78541548j
 ... -2.71590004+2.85962026j  1.11371932-1.02837059j
 -0.90020355-0.95577982j]
[[-2.90141193  2.63064032]
 [-2.81521769 -1.06264591]
 [ 1.02505341 -0.78541548]
 ...
 [-2.71590004  2.85962026]
 [ 1.11371932 -1.02837059]
 [-0.90020355 -0.95577982]]
[[[ 1. -1.]]

 [[-1. -1.]]

 [[-3. -1.]]

 ...

 [[ 1. -1.]]

 [[-3. -1.]]

 [[-1. -3.]]]


In [11]:
for batch_x, batch_y in data.take(5):
    print(batch_x, batch_y)

tf.Tensor(
[[-0.80201015 -2.91296718]
 [-2.77485051 -1.12735148]
 [ 1.02503486  0.9946743 ]
 [-2.76281307 -3.04943822]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[-1.  3.]]

 [[-1. -1.]]

 [[ 3.  1.]]

 [[-1.  1.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[ 0.81424623 -0.67649034]
 [-0.83270232 -0.64406755]
 [-2.76197069 -2.89895599]
 [-1.29783767  2.89556745]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[-3. -1.]]

 [[-1. -3.]]

 [[-1.  1.]]

 [[ 1. -3.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[-0.89285223  0.86169173]
 [-2.93400753 -1.16189352]
 [-3.15946925  2.5858418 ]
 [-2.87998551  1.99195784]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[ 1.  3.]]

 [[-1. -1.]]

 [[ 1. -1.]]

 [[ 1. -1.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[ 3.34787792 -2.81852136]
 [-1.03504781 -0.99932729]
 [-1.0756993   2.62979663]
 [ 0.85236448 -2.84770367]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[-3.  3.]]

 [[-1. -3.]]

 [[ 1. -3.]]

 [[-3.  1.]]], shape=(4, 1, 2), dtype=float64)


In [12]:
# Note: If you are planning on calling multiple time,
# you can user the iterator way:
ite_data = iter(data)
for i in range(5):
    batch_x, batch_y = next(ite_data)
    print(batch_x, batch_y)

tf.Tensor(
[[ 2.88953709 -2.83490263]
 [-2.82248642 -3.00900188]
 [-0.7992982  -2.89378703]
 [ 2.66566396 -0.96587498]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[-3.  3.]]

 [[-1.  1.]]

 [[-1.  3.]]

 [[-3. -3.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[ 2.60785085  2.62611939]
 [-2.62864549  0.84292861]
 [-3.06476194  0.7531055 ]
 [ 3.12939683 -0.67741472]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[ 3. -3.]]

 [[ 1.  1.]]

 [[ 1.  1.]]

 [[-3. -3.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[ 1.10689668 -0.96252452]
 [ 1.13275178 -1.06150124]
 [-2.79262616  0.47827914]
 [ 2.74075481  1.21184594]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[-3. -1.]]

 [[-3. -1.]]

 [[ 1.  1.]]

 [[ 3.  3.]]], shape=(4, 1, 2), dtype=float64)
tf.Tensor(
[[-1.29783767  2.89556745]
 [ 3.34787792 -2.81852136]
 [ 2.72080222  2.73953054]
 [-0.84898578  0.90437721]], shape=(4, 2), dtype=float64) tf.Tensor(
[[[ 1. -3.]]

 [[-3.  3.]]

 [[ 3. -3.]]

 [[ 1.  3.]]], shape=(4, 1, 2), dtype=float64)


In [13]:
# Weight and Bias, initialized randomly.
W = tf.Variable(rng.randn(), name="weight")
b = tf.Variable(rng.randn(), name="bias")

# Linear regression (Wx + b).
def linear_regression(x):
    return W * x + b

# Mean square error.
def mean_square(y_pred, y_true):
    return tf.reduce_mean(tf.square(y_pred - y_true))

# Stochastic Gradient Descent Optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [14]:
# Optimization process. 
def run_optimization():
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        pred = linear_regression(data_ri_x)
        loss = mean_square(pred, source_ri_x)

    # Compute gradients.
    gradients = g.gradient(loss, [W, b])
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [ ]:
# Run training for the given number of steps.
for step in range(1, training_steps + 1):
    # Run the optimization to update W and b values.
    run_optimization()
    
    if step % display_step == 0:
        pred = linear_regression(data_ri_x)
        loss = mean_square(pred, source_ri_x)
        print("step: %i, loss: %f, W: %f, b: %f" % (step, loss, W.numpy(), b.numpy()))